In [ ]:
!pip install beautifulsoup4
!pip install requests
!pip install pandas

In [1]:
from bs4 import BeautifulSoup
import requests

In [33]:
url='https://egov.kz/cms/ru/online-services/for_citizen'
headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
            }
def request_sender(url):
    res = requests.get(url, headers=headers)
    res.raise_for_status()
    return res
    

In [3]:
soup = BeautifulSoup(request_sender(url).text, 'lxml')

In [34]:
def info_getter(service):
    req2 = request_sender('https://egov.kz'+service)
    soup2 = BeautifulSoup(req2.content, 'lxml')
    info_block = soup2.find_all(class_='info-block-horizontal')
    if len(info_block)<2: return 'Empty'
    info_block = info_block[1].find_all('p')
    content=''
    for text in info_block:
        content+=text.text.strip()+'\n'
    return content

In [ ]:
len(soup.find_all('div', class_='category-item'))

In [ ]:
services = []
services_html = soup.find_all('div', class_='category-item')
for service in services_html:
    category = service.h2.text.replace("\n","").replace(" ","")
    for link in service.ul.find_all('li'):
        print(link.a['href'])
        info = info_getter(link.a['href'])
        services.append({'category': category,
                         'service' : link.text,
                         'link' : link.a['href'],
                         'info': info})
    print(f"{category}")

In [40]:
import pandas as pd
df = pd.DataFrame(services)
df
df.to_csv('egov_parsed.csv')

In [7]:
# df = pd.read_csv('./egov_parsed.csv', index_col=0) 
# uncomment if you want to load from csv

In [4]:
def search(df, column, word):
    df = df[df[column].str.contains(word, case=False, na=False)]
    return df

In [8]:
search(df, 'service', 'брак')

,category,service,link,info
6,Семья,Государственная регистрация заключения брака (...,/cms/ru/online-services/for_citizen/pass33-4_mid,Результат оказания услуги\nСвидетельство о зак...
7,Семья,Государственная регистрация расторжения брака ...,/cms/ru/online-services/for_citizen/pass33-5_mid,Результат оказания услуги\nСвидетельство о рас...
9,Семья,Заявление на заключение брака/регистрация брака,/cms/ru/online-services/for_citizen/e_027,Результат оказания услуги\nСвидетельство о гос...
17,Семья,Подача заявки на повторное свидетельство о зак...,/cms/ru/online-services/for_citizen/e_021,Результат оказания услуги\nПовторное свидетель...
18,Семья,Подача заявки на повторное свидетельство о рас...,/cms/ru/online-services/for_citizen/e_023,Результат оказания услуги\nПовторное свидетель...
21,Семья,Получение справки о заключении брака,/cms/ru/online-services/for_citizen/e_019,Результат оказания услуги\nПовторное свидетель...
22,Семья,Получение справки о расторжении брака,/cms/ru/online-services/for_citizen/e_018,Результат оказания услуги\nПовторное свидетель...
30,Семья,Расторжение брака,/cms/ru/online-services/for_citizen/pass017_mu,Результат оказания услуги\nСвидетельство о гос...


In [9]:
df.shape

(266, 4)